# Investigating large forces in HMC

In [ ]:
from math import pi as π, sqrt

import torch
import pandas as pd
import seaborn as sns
import numpy as np

from nflows_xy.core import FlowBasedSampler, PullbackAction
from nflows_xy.flows import AutoregressiveFlow, DummyFlow
from nflows_xy.plot import plot_training_metrics
from nflows_xy.train import train, test
from nflows_xy.xy import action as Action
from nflows_xy.utils import mod_2pi

Tensor = torch.Tensor

## Data generation

In [ ]:
@torch.no_grad()
def _leapfrog(
    z0,
    p0,
    action,
    step_size,
    traj_length,
):
    """Similar to nflows_xy.hmc.leapfrog but tracks quantities at each step."""

    n_steps = max(1, round(traj_length / abs(step_size)))

    z = z0.clone()
    p = p0.clone()
    ε = step_size
    F = action.grad(z).negative()

    trajectory = [z.clone()]

    for _ in range(n_steps):     
       
        p = p + (ε / 2) * F
        z = mod_2pi(z + ε * p)
        F = action.grad(z).negative()
        p = p + (ε / 2) * F
        
        trajectory.append(z.clone())

    return torch.cat(trajectory)

def _make_data(z: Tensor, action: PullbackAction) -> pd.DataFrame:
        
    with torch.no_grad():
        φ, ldj = action.flow(z)

    S_pull = action(z)
    F = action.grad(z).negative()
    S_targ = action.target(φ)

    data = torch.cat(
        [
            z.squeeze(-1),
            φ.squeeze(-1),
            F.squeeze(-1),
            F.pow(2).sum(1).sqrt(),
            ldj,
            S_pull,
            S_targ,
        ],
        dim=1,
    )
    columns = [
        *[f"z{i}" for i in range(L)],
        *[f"φ{i}" for i in range(L)],
        *[f"F{i}" for i in range(L)],
        "|F|",
        "ldj",
        "S_pull",
        "S_targ",
    ]
    
    data = pd.DataFrame(
        data=data,
        columns=columns,
    )

    return data

def make_scatter(action: PullbackAction, size: int, seed: int | None = None) -> pd.DataFrame:
    seed = seed or torch.seed()
    print(f"Seeding RNG with seed {seed}")
    torch.manual_seed(seed)
    L = action.lattice_size
    z = torch.empty(size, L, 1).uniform_(0, 2 * π)
    return _make_data(z, action)

def make_linspace(action: PullbackAction, size: int) -> pd.DataFrame:
    assert action.lattice_size == 2
    z = torch.stack(
        [
            torch.zeros(size),
            torch.linspace(0, 2 * π, size),
        ],
        dim=1,
    ).unsqueeze(-1)
    return _make_data(z, action)

def make_trajectory(action: PullbackAction, step: float, length: float = 1.0, seed: int | None = None) -> pd.DataFrame:
    seed = seed or torch.seed()
    print(f"Seeding RNG with seed {seed}")
    torch.manual_seed(seed)
    L = action.lattice_size
    z = _leapfrog(
        z0=torch.empty(1, L, 1).uniform_(0, 2 * π),
        p0=torch.empty(1, L, 1).normal_(),
        action=action,
        step_size=step,
        traj_length=length,
    )
    return _make_data(z, action)

## Hybrid Monte Carlo

The action here is

$$
    S = -2\beta \cos(\phi_0 - \phi_1) 
$$

The largest possible magnitude force is hence

$$
    |F_\text{max}| = 2\sqrt{2} \beta
$$

In [ ]:
L = 2

target = Action(beta=3.0, lattice_size=L, lattice_dim=1)
flow = DummyFlow()
model = FlowBasedSampler(flow, target)

pullback = PullbackAction(model.flow, model.target)
trajectory = make_trajectory(pullback, step=0.01, length=4.0, seed=123456789)
axes = trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

axes[-1].axhline(2 * sqrt(2) * target.beta, ls="--")  # max force

## Demonstration of the problem

In [ ]:
L = 2

target = Action(beta=3.0, lattice_size=L, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=L,
    n_mixture=12,
    net_shape=[32],
    net_activation="Tanh",
)
model = FlowBasedSampler(flow, target)

training_metrics = train(
    model, 
    n_steps=2000,
    batch_size=2048,
)

training_metrics.plot(x="step", y=["loss", "ess", "vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(model.flow, model.target)
data = make_linspace(pullback, size=10000)
data.plot(x="z1", y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
data = data[abs(data.z1 - π - 0.15) < 0.1]
data.plot(x="z1", y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
pullback = PullbackAction(model.flow, model.target)
trajectory = make_trajectory(pullback, step=0.01, length=4.0, seed=123456789)

trajectory.plot(y=["z1", "φ1"], marker=".", linestyle=":")

In [ ]:
trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
data = make_scatter(pullback, size=10000)

data.plot.scatter("z0", "z1", c="ldj")
data.plot.scatter("φ0", "φ1", c="ldj")
data.plot.scatter("S_targ", "S_pull", c="|F|")

## Modifications that do not solve the problem

### Remove the neural network

In [ ]:
L = 2

target = Action(beta=3.0, lattice_size=L, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=L,
    n_mixture=12,
    net_shape=[],
    net_activation=None,
)
model = FlowBasedSampler(flow, target)

training_metrics = train(
    model, 
    n_steps=2000,
    batch_size=2048,
)

training_metrics.plot(x="step", y=["loss", "ess", "vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(model.flow, model.target)
data = make_linspace(pullback, size=10000)
data.plot(x="z1", y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
trajectory = make_trajectory(pullback, step=0.01, length=4.0, seed=123456789)
trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
data = make_scatter(pullback, size=10000)

data.plot.scatter("z0", "z1", c="ldj")
data.plot.scatter("φ0", "φ1", c="ldj")
data.plot.scatter("S_targ", "S_pull", c="|F|")

### Simplify transformation

In [ ]:
L = 2

target = Action(beta=3.0, lattice_size=L, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=L,
    n_mixture=1,
    net_shape=[16],
    net_activation="Tanh",
)
model = FlowBasedSampler(flow, target)

training_metrics = train(
    model, 
    n_steps=2000,
    batch_size=2048,
)

training_metrics.plot(x="step", y=["loss", "ess", "vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(model.flow, model.target)
data = make_linspace(pullback, size=10000)
data.plot(x="z1", y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
pullback = PullbackAction(model.flow, model.target)
trajectory = make_trajectory(pullback, step=0.01, length=4.0, seed=987654321)

trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
data = make_scatter(pullback, size=10000)

data.plot.scatter("z0", "z1", c="ldj")
data.plot.scatter("φ0", "φ1", c="ldj")
data.plot.scatter("S_targ", "S_pull", c="|F|")

### Simplify transformation AND remove neural network

In [ ]:
L = 2

target = Action(beta=3.0, lattice_size=L, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=L,
    n_mixture=1,
    net_shape=[],
    net_activation="Identity",
)
model = FlowBasedSampler(flow, target)

training_metrics = train(
    model, 
    n_steps=2000,
    batch_size=2048,
)

training_metrics.plot(x="step", y=["loss", "ess", "vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(model.flow, model.target)
data = make_linspace(pullback, size=10000)
data.plot(x="z1", y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
pullback = PullbackAction(model.flow, model.target)
trajectory = make_trajectory(pullback, step=0.01, length=4.0, seed=987654321)

trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
data = make_scatter(pullback, size=10000)

data.plot.scatter("z0", "z1", c="ldj")
data.plot.scatter("φ0", "φ1", c="ldj")
data.plot.scatter("S_targ", "S_pull", c="|F|")

### Training at lower beta

In [ ]:
L = 2

target = Action(beta=1.0, lattice_size=L, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=L,
    n_mixture=12,
    net_shape=[16],
    net_activation="Tanh",
)
model = FlowBasedSampler(flow, target)

training_metrics = train(
    model, 
    n_steps=2000,
    batch_size=2048,
)

training_metrics.plot(x="step", y=["loss", "ess", "vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(model.flow, model.target)
data = make_linspace(pullback, size=10000)
data.plot(x="z1", y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
pullback = PullbackAction(model.flow, model.target)
trajectory = make_trajectory(pullback, step=0.01, length=8.0, seed=14850734071541236791)

trajectory.plot(y=["z1", "φ1"], marker=".", linestyle=":")

trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
new_target = Action(beta=3., lattice_size=L, lattice_dim=1)

pullback = PullbackAction(model.flow, new_target)
data = make_linspace(pullback, size=10000)
data.plot(x="z1", y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
new_target = Action(beta=3., lattice_size=L, lattice_dim=1)

pullback = PullbackAction(model.flow, new_target)
trajectory = make_trajectory(pullback, step=0.01, length=8.0, seed=14850734071541236791)

trajectory.plot(y=["φ0", "z1", "φ1"], marker=".", linestyle=":")

trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

### Increasing the amount of training

In [ ]:
L = 2

target = Action(beta=3.0, lattice_size=L, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=L,
    n_mixture=18,
    net_shape=[8],
    net_activation="Tanh",
)
model = FlowBasedSampler(flow, target)

training_metrics = train(
    model, 
    n_steps=2000,
    batch_size=10000,
)

training_metrics["log_1m_ess"] = np.log(1 - training_metrics["ess"])
training_metrics["log_vlw"] = np.log(training_metrics["vlw"])
training_metrics.plot(x="step", y=["log_1m_ess", "log_vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(model.flow, model.target)
trajectory = make_trajectory(pullback, step=0.01, length=1.0)

trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

# Weird: ESS reaches maximum then decreases

Probably a symptom of a very inflexible transformation.

In [ ]:
L = 2

target = Action(beta=3.0, lattice_size=L, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=L,
    n_mixture=1,
    net_shape=[],
    net_activation="Identity",
)
model = FlowBasedSampler(flow, target)

training_metrics = train(
    model, 
    n_steps=4000,
    batch_size=2048,
    init_lr=1e-3
)

training_metrics.plot(x="step", y=["loss", "ess", "vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(model.flow, model.target)
trajectory = make_trajectory(pullback, step=0.01, length=4.0)

trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
data = make_scatter(pullback, size=10000)

data.plot.scatter("z0", "z1", c="ldj")
data.plot.scatter("φ0", "φ1", c="ldj")
data.plot.scatter("S_targ", "S_pull", c="|F|")